In [97]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import StaleElementReferenceException
import time

# Safari WebDriver 인스턴스 생성
driver = webdriver.Safari()


products_data = {}

try:
    # 디스코드 로그인 페이지 로드
    driver.get('https://search.danawa.com/dsearch.php?query=%EA%B7%B8%EB%A6%B0%ED%82%A4%EC%9C%84+1kg&originalQuery=%EA%B7%B8%EB%A6%B0%ED%82%A4%EC%9C%84+1kg&previousKeyword=%EA%B7%B8%EB%A6%B0%ED%82%A4%EC%9C%84+1kg&checkedInfo=N&volumeType=allvs&page=1&limit=40&sort=priceASC&list=list&boost=true&tab=goods&addDelivery=Y&coupangMemberSort=N&isInitTireSmartFinder=N&recommendedSort=N&defaultUICategoryCode=16325265&defaultPhysicsCategoryCode=46803%7C46818%7C50379%7C0&defaultVmTab=10&defaultVaTab=387&isZeroPrice=Y&quickProductYN=N&priceUnitSort=N&priceUnitSortOrder=A')

    # 충분한 로딩 시간 대기
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".product_list")))
    main_window = driver.current_window_handle  # 메인 윈도우 핸들 저장
    
    # Find all products within the product list that have the class '.prod_item .searched'
    products = driver.find_elements(By.CSS_SELECTOR, ".product_list .prod_item.searched")
    print(len(products))

    for product in products:
        try:
        # Navigate to the main information of each product to access the price list
            main_info = product.find_element(By.CSS_SELECTOR, ".prod_main_info")
            price_list = main_info.find_element(By.CSS_SELECTOR, ".prod_pricelist")

            # Extract the 'alt' text from the mall icon
            mall_icon_alt_text = price_list.find_element(By.CSS_SELECTOR, ".mall_icon img").get_attribute('alt')
            print("Mall Icon Alt Text:", mall_icon_alt_text)

            # Extract the price text
            price_text = price_list.find_element(By.CSS_SELECTOR, ".price_sect strong").text
            print("Price:", price_text)

            # Click on the link with class '.click_log_product_searched_price_'
            click_link = price_list.find_element(By.CSS_SELECTOR, ".click_log_product_searched_price_")
            driver.execute_script("arguments[0].click();", click_link)
            time.sleep(5)
        
            # Wait a bit for the action to complete or for the new page to load (if necessary)
                    # 현재 열려 있는 창/탭의 핸들을 모두 가져옴
            windows = driver.window_handles

            # 새 창으로 포커스를 전환 (가장 최근에 열린 창으로 가정)
            driver.switch_to.window(windows[-1])

            # WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".")))  # 메인 윈도우가 재로드 됐다면, 대기

            time.sleep(5)
            # 새 창의 URL 가져오기
            current_url = driver.current_url
            print("New Window URL:", current_url)

            # 필요한 작업 수행 후, 창을 닫고 원래 창으로 돌아가기
            driver.close()

            time.sleep(1)

            driver.switch_to.window(main_window)  # 메인 윈도우로 전환
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".product_list")))  # 메인 윈도우가 재로드 됐다면, 대기

            time.sleep(3)
        except StaleElementReferenceException:
            # 요소가 더 이상 유효하지 않을 때 예외 처리
            print("StaleElementReferenceException detected. Moving to the next product.")
            continue  # 다음 반복으로 넘어감
        

finally:
    # WebDriver 종료
    driver.quit()

KeyboardInterrupt: 

In [98]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
from urllib.parse import urlparse, parse_qs, urlencode, urlunparse, quote


def crawlingData(page, search_keyword):
    # Safari WebDriver 인스턴스 생성
    driver = webdriver.Safari()

    try:
        # 페이지 로드
        url = f'https://search.danawa.com/dsearch.php?query={search_keyword}&checkedInfo=N&volumeType=allvs&page={page}&limit=40&sort=priceASC&list=list&boost=true&tab=main&addDelivery=Y&coupangMemberSort=N&isInitTireSmartFinder=N&recommendedSort=N&defaultUICategoryCode=16325265&defaultPhysicsCategoryCode=46803%7C46818%7C50379%7C0&defaultVmTab=10&defaultVaTab=389&isZeroPrice=Y&quickProductYN=N&priceUnitSort=Y&priceUnitSortOrder=A'
        driver.get(url)
        # driver.get(f'https://search.danawa.com/dsearch.php?query={search_keyword}&checkedInfo=N&volumeType=allvs&page={page}&limit=40&sort=priceASC&list=list&boost=true&tab=goods&addDelivery=Y&coupangMemberSort=N&isInitTireSmartFinder=N&recommendedSort=N&defaultUICategoryCode=16325265&defaultPhysicsCategoryCode=46803%7C46818%7C50379%7C0&defaultVmTab=10&defaultVaTab=387&isZeroPrice=Y&quickProductYN=N&priceUnitSort=N&priceUnitSortOrder=A')

        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".product_list")))
        main_window = driver.current_window_handle  # 메인 윈도우 핸들 저장
        
        products = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".product_list .prod_item.searched")))

        print(len(products))

        for index, product in enumerate(products[:5]):
            try:
                # 새로운 요소 참조를 얻기 위해 매번 elements를 다시 조회합니다.
                product = driver.find_elements(By.CSS_SELECTOR, ".product_list .prod_item.searched")[index]
                main_info = product.find_element(By.CSS_SELECTOR, ".prod_main_info")
                price_list = main_info.find_element(By.CSS_SELECTOR, ".prod_pricelist")

                mall_icon_alt_text = price_list.find_element(By.CSS_SELECTOR, ".mall_icon img").get_attribute('alt').strip()
                print("Mall Icon Alt Text:", mall_icon_alt_text)

                price_text = price_list.find_element(By.CSS_SELECTOR, ".price_sect strong").text
                print("Price:", price_text)

                # 새 창에서 작업
                click_link = price_list.find_element(By.CSS_SELECTOR, ".click_log_product_searched_price_")
                driver.execute_script("arguments[0].click();", click_link)
                
                # 새 창이 나타날 때까지 기다립니다.
                WebDriverWait(driver, 10).until(EC.number_of_windows_to_be(2))
                windows = driver.window_handles
                new_window = [window for window in windows if window != main_window][0]
                driver.switch_to.window(new_window)
                
                # # 새 창에서 필요한 정보를 가져온 후 닫기
                if mall_icon_alt_text == "티몬":


                    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".partner_info")))  # 새 창에서 콘텐츠가 로드될 때까지 대기

                    # shop_content = driver.find_elements(By.CSS_SELECTOR, "body")

                    partner_info = driver.find_element(By.CSS_SELECTOR, ".partner_info")
                    print(partner_info)

                    partner_button_click = driver.find_element(By.CSS_SELECTOR, ".partner_info button")
                    driver.execute_script("arguments[0].click();", partner_button_click)

                    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".partner_info .slide-ct")))  # 새 창에서 콘텐츠가 로드될 때까지 대기

                    partner_info_after_click = driver.find_element(By.CSS_SELECTOR, ".partner_info .slide-ct")
                    product_title = driver.find_element(By.CSS_SELECTOR, ".deal_title_main").text.strip()  

                    biz_name = partner_info_after_click.find_element(By.CSS_SELECTOR, "tr:nth-child(1) > td").text.strip()  
                    biz_email = partner_info_after_click.find_element(By.CSS_SELECTOR, "tr:nth-child(6) > td").text.strip()  
                    biz_phone = partner_info_after_click.find_element(By.CSS_SELECTOR, "tr:nth-child(7) > td").text.strip()  

                    print(product_title, biz_name,biz_email, biz_phone)

                    current_url = driver.current_url
                    print("New Window URL:", current_url)
                elif mall_icon_alt_text == "쿠팡":
                    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".prod-atf-main")))  
                    current_url = driver.current_url
                    print("New Window URL:", current_url)
                    #id contents
                elif mall_icon_alt_text == "옥션":
                    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".vip-content")))  
                    current_url = driver.current_url
                    print("New Window URL:", current_url)
                elif mall_icon_alt_text == "위메프플러스":
                    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".goods_wrap")))  
                    current_url = driver.current_url
                    print("New Window URL:", current_url)
                elif mall_icon_alt_text == "위메프":
                    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".container")))  
                    current_url = driver.current_url
                    print("New Window URL:", current_url)
                elif mall_icon_alt_text == "롯데ON":
                    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".productWrap")))  
                    current_url = driver.current_url
                    print("New Window URL:", current_url)
                elif mall_icon_alt_text == "G마켓":
                    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".item-topinfowrap")))  
                    current_url = driver.current_url
                    print("New Window URL:", current_url)
                elif mall_icon_alt_text == "11번가":
                    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".l_product_summary")))  
                    current_url = driver.current_url
                    print("New Window URL:", current_url)
                elif mall_icon_alt_text == "이마트인터넷쇼핑몰":
                    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".cdtl_row_top")))  
                    current_url = driver.current_url
                    print("New Window URL:", current_url)
                elif mall_icon_alt_text == "SSG.COM":
                    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".content_primary")))  
                    current_url = driver.current_url
                    print("New Window URL:", current_url)
                elif mall_icon_alt_text == "인터파크":
                    prev_popup_window = driver.current_window_handle  # 메인 윈도우 핸들 저장

                    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".productInfomation")))  
                    windows = driver.window_handles
                    new_window = next(window for window in windows if window != prev_popup_window and window != main_window)
                    driver.switch_to.window(new_window)
                    driver.close()
                    driver.switch_to.window(prev_popup_window)
                    current_url = driver.current_url
                    print("New Window URL:", current_url)
                elif mall_icon_alt_text == "롯데홈쇼핑":
                    prev_popup_window = driver.current_window_handle  # 메인 윈도우 핸들 저장

                    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".detail_sec")))  
                    windows = driver.window_handles
                    new_window = next(window for window in windows if window != prev_popup_window and window != main_window)
                    driver.switch_to.window(new_window)
                    driver.close()
                    driver.switch_to.window(prev_popup_window)
                    current_url = driver.current_url
                    print("New Window URL:", current_url)


                
                
                

                driver.close()
                driver.switch_to.window(main_window)  # 메인 윈도우로 전환

                # 원래 윈도우로 돌아온 후 필요한 웹 요소를 다시 찾습니다.
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".product_list")))


            except StaleElementReferenceException:
                continue

    except StaleElementReferenceException:
        print(StaleElementReferenceException)

    finally:
        driver.quit()

if __name__ == "__main__":
    crawlingData(1, "그린키위 1kg")


36
Mall Icon Alt Text: 티몬
Price: 6,960
<selenium.webdriver.remote.webelement.WebElement (session="451D8FAB-25CF-45C9-9FAF-185EFD3FBB7F", element="node-C38BAD7B-639C-44CB-893C-D4428A6447F1")>
제스프리 썬 골드키위/그린키위 특대과 외 사이즈별 골라담기 주식회사 오렌지씨 orangec1009@naver.com 010-9177-2302티몬 고객센터 | 1544-6240
New Window URL: https://www.tmon.co.kr/deal/14431890826?opt_deal_srl=14487448682&coupon_srl=3383202&utm_source=danawa&utm_medium=affiliate&utm_term=205009_%EB%8B%A4%EB%82%98%EC%99%80DB&utm_content=&utm_campaign=%EB%8B%A4%EB%82%98%EC%99%80
Mall Icon Alt Text: 쿠팡
Price: 7,210
New Window URL: https://www.coupang.com/vp/products/5750079900?itemId=15060092714&vendorItemId=82282468174&src=1032034&spec=10305197&addtag=400&ctag=5750079900&lptag=I15060092714&itime=20240325225441&pageType=PRODUCT&pageValue=5750079900&wPcid=17113748811018626266258&wRef=prod.danawa.com&wTime=20240325225441&redirect=landing&mcid=fe46fb07c67746b2a7188d89093fb9c7
Mall Icon Alt Text: 티몬
Price: 8,720
<selenium.webdriver.remote.webeleme

36
Mall Icon Alt Text: 티몬
Price: 6,960


NoSuchElementException: Message: ; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.keys import Keys
import time


def wait_for_page_load(driver, timeout=30):
    old_page = driver.find_element(By.TAG_NAME, 'html')
    yield
    WebDriverWait(driver, timeout).until(
        EC.staleness_of(old_page)
    )

def check_page_is_loaded(driver, timeout=30):
    # 페이지가 완전히 로드될 때까지 기다림 (document.readyState가 complete가 될 때까지)
    WebDriverWait(driver, timeout).until(
        lambda d: d.execute_script('return document.readyState') == 'complete'
    )

# Safari WebDriver 인스턴스 생성
driver = webdriver.Safari()

try:
    # 페이지 로드
    # driver.get('https://prod.danawa.com/bridge/go_link_goods.php?cate_c1=46803&cate_c2=46818&cate_c3=50379&cate_c4=0&cmpny_c=TP40F&link_prod_c=I15060092714&keyword=%EA%B7%B8%EB%A6%B0%ED%82%A4%EC%9C%84+1kg&r=17113687248228&cate=16325265')
    driver.get('https://prod.danawa.com/bridge/go_link_goods.php?cate_c1=46803&amp;cate_c2=46818&amp;cate_c3=50379&amp;cate_c4=0&amp;cmpny_c=TP40F&amp;link_prod_c=I15060092714&amp;keyword=%EA%B7%B8%EB%A6%B0%ED%82%A4%EC%9C%84+1kg&amp;r=17113687248228&amp;cate=16325265')
               

    # WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".product_list")))
    # main_window = driver.current_window_handle  # 메인 윈도우 핸들 저장
    
    # products = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".product_list .prod_item.searched")))

    # print(len(products))

    # for index, product in enumerate(products):
    time.sleep(10)
       
finally:
    driver.quit()





In [ ]:
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.oauth2.credentials import Credentials
from google.oauth2 import service_account
from google.auth.transport.requests import Request
import os.path
import json
import pickle
import requests

def oauth():
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets', 
              'https://www.googleapis.com/auth/script.projects', 'https://www.googleapis.com/auth/drive', 'https://www.googleapis.com/auth/drive.file'
              ]
    creds_filename = 'test-daebong-service-account.json'  # 서비스 계정 파일 경로를 지정합니다.

    # 서비스 계정 파일을 사용하여 인증 정보를 로드합니다.
    creds = service_account.Credentials.from_service_account_file(creds_filename, scopes=SCOPES)
    return creds

def oauthByWeb():
    # 필요한 스코프를 지정합니다.
    SCOPES = [
        'https://www.googleapis.com/auth/spreadsheets',
        'https://www.googleapis.com/auth/script.projects',
        'https://www.googleapis.com/auth/drive',
        'https://www.googleapis.com/auth/drive.file'
    ]
    creds = None
    # 'token.json' 파일이 존재하면, 저장된 인증 정보를 불러옵니다.
    if os.path.exists('token.json'):
        with open('token.json', 'rb') as token:
            creds = pickle.load(token)

    # 저장된 인증 정보가 없거나, 유효하지 않은 경우 새로운 인증을 진행합니다.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            creds_filename = 'oauth-new-daebong.json'
            flow = InstalledAppFlow.from_client_secrets_file(creds_filename, SCOPES)
            creds = flow.run_local_server(port=8080)
        # 새로운 인증 정보를 'token.json'에 저장합니다.
        with open('token.json', 'wb') as token:
            pickle.dump(creds, token)

    return creds



In [ ]:
def append_crawling_data(service, spreadsheet_id, values, sheet_name):
    range_name = f'{sheet_name}!b2:D'


    body = {
    'values': values    
    }

    # 스프레드시트에 데이터 쓰기
    result = service.spreadsheets().values().update(
    spreadsheetId=spreadsheet_id, range=range_name,
    valueInputOption='RAW', body=body).execute()

    print(result)

In [ ]:
# unit_pricing_spreadsheet_id = "1oxak1ZgQi3YhbC5UKPcmlrjV9ekMt53BVkHlycBW3uQ"
#크롤링 데이터 원본
unit_pricing_spreadsheet_id = "132EosVTtzTuqNrfJ2mDxRD-TbOHxDGdGXKwpgA92y54"
worksheet_name = '캄므커머스_데이터원본'
arranged_worksheet_name = '캄므커머스'

if __name__ == "__main__":
    creds = oauthByWeb()
    service_sheets = build('sheets', 'v4', credentials=creds)
    append_crawling_data(service_sheets, unit_pricing_spreadsheet_id,product_processed_data, worksheet_name)



{'spreadsheetId': '132EosVTtzTuqNrfJ2mDxRD-TbOHxDGdGXKwpgA92y54', 'updatedRange': "'캄므커머스_데이터원본'!B2:D220", 'updatedRows': 219, 'updatedColumns': 3, 'updatedCells': 657}
